In [1]:
import pdb, os, argparse, itertools
import pandas as pd
import numpy as np
from pymongo import InsertOne, DeleteOne
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from kdutils.mongodb import MongoDBManager
from lib.aux001 import *

In [3]:
code='ims'
method = 'aicso0'

In [5]:
total_data, total_returns = fetch_market(instruments=code, method=method, 
                                          name=['train','val','test'])

./records/aicso0/ims/basic/train_data.feather
./records/aicso0/ims/basic/val_data.feather
./records/aicso0/ims/basic/test_data.feather
./records/aicso0/ims/returns/train_returns.feather
./records/aicso0/ims/returns/val_returns.feather
./records/aicso0/ims/returns/test_returns.feather


In [27]:
factors_data1 = calc_expression("MCPS(5,'ixy013_5_10_1') * MCPS(5,'ixy013_5_10_1')",total_data.set_index('trade_time'))
#factors_data1 = factors_data1
factors_data1 = factors_data1.set_index(['trade_time','code'])
factors_data1.groupby('code').apply(lambda x: x['transformed'].autocorr(lag=1)).mean()

0.7836713324457943

In [30]:
#factors_data1
total_returns = total_returns.set_index(['trade_time','code'])

In [36]:
period =  10

In [37]:
time_index = factors_data1.index.get_level_values('trade_time')
mask = time_index.minute % period == 0
factors_filtered = factors_data1[mask]
factors_filtered

,,transformed
trade_time,code,
2022-07-25 09:40:00,IM,0.016314
2022-07-25 09:50:00,IM,0.006561
2022-07-25 10:00:00,IM,0.001575
2022-07-25 10:10:00,IM,0.051058
2022-07-25 10:20:00,IM,0.033924
...,...,...
2025-03-14 14:20:00,IM,0.011350
2025-03-14 14:30:00,IM,0.011717
2025-03-14 14:40:00,IM,0.414853


In [38]:
time_index = total_returns.index.get_level_values('trade_time')
mask = time_index.minute % period == 0
returns_filtered = total_returns[mask]
returns_filtered

,,nxt1_ret_1h,nxt1_ret_2h,nxt1_ret_3h,nxt1_ret_5h,nxt1_ret_10h,nxt1_ret_15h,time_weight,equal_weight
trade_time,code,,,,,,,,
2022-07-22 09:30:00,IM,0.001999,0.003928,0.003025,0.003425,0.003205,-0.000429,0.002813,0.002984
2022-07-22 09:40:00,IM,0.000148,0.000192,-0.001929,-0.003634,-0.010992,-0.010184,-0.000184,-0.000530
2022-07-22 09:50:00,IM,-0.000748,-0.000632,0.000483,0.000808,0.000382,0.002189,-0.000504,-0.000299
2022-07-22 10:00:00,IM,0.000529,-0.000129,-0.000440,0.001807,0.002413,0.001820,0.000148,-0.000014
2022-07-22 10:10:00,IM,0.000104,0.000630,0.000284,-0.000594,0.001462,0.002661,0.000310,0.000340
...,...,...,...,...,...,...,...,...,...
2025-03-14 14:30:00,IM,-0.000088,0.000395,0.000052,-0.000109,-0.000991,-0.000641,0.000096,0.000120
2025-03-14 14:40:00,IM,0.000216,0.000699,0.000587,0.000350,0.000370,0.002116,0.000439,0.000501
2025-03-14 14:50:00,IM,0.000183,0.000730,0.001330,0.001746,0.002091,-0.000259,0.000556,0.000748


In [39]:
returns_filtered.groupby('code').apply(lambda x: x['nxt1_ret_{0}h'.format(period)].autocorr(lag=1)).mean()

0.02613588485299884

In [40]:
factors_filtered.groupby('code').apply(lambda x: x['transformed'].autocorr(lag=1)).mean()

0.023657397437204578

In [44]:
total_data = factors_filtered.reset_index().merge(returns_filtered.reset_index(),
                                     on=['trade_time','code'])
total_data.head()

,trade_time,code,transformed,nxt1_ret_1h,nxt1_ret_2h,nxt1_ret_3h,nxt1_ret_5h,nxt1_ret_10h,nxt1_ret_15h,time_weight,equal_weight
0,2022-07-25 09:40:00,IM,0.016314,-0.002059,-0.001321,-0.001599,-0.001167,-0.005412,-0.006423,-0.001736,-0.001660
1,2022-07-25 09:50:00,IM,0.006561,-0.001346,-0.000950,0.000047,-0.001011,-0.002654,0.000077,-0.000982,-0.000750
2,2022-07-25 10:00:00,IM,0.001575,0.000662,0.000236,0.001359,0.002731,0.000366,0.000903,0.000636,0.000752
3,2022-07-25 10:10:00,IM,0.051058,-0.000102,-0.000014,-0.000264,0.000537,-0.001119,0.002756,-0.000100,-0.000127
4,2022-07-25 10:20:00,IM,0.033924,0.001176,0.002340,0.003014,0.003875,0.002865,0.003651,0.001871,0.002177


In [51]:
basic_data,perf_data = calc_evaluate(factor_data=total_data, 
                    ret_name='nxt1_ret_{0}h'.format(period), 
                    roll_win=2, scale_method='roll_min_max', fee=0,
                    is_plot=True)

ValueError: min_periods 5 must be <= window 2